# Classificazione globale

In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 67.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=34818584fdba5bb4a3a538b5076b21940ae515f7774bd0ebaf8e9849db7bed39
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark import SparkContext
# initialize a new Spark Context to use for the execution of the script
sc = SparkContext(appName="MY-APP-NAME", master="local[*]")

In [ ]:
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
filename = '/content/drive/MyDrive/DDAM/Australia Rain/rain_tomorrow_australia.csv'

Mounted at /content/drive/


In [ ]:
# carico il dataframe
df = sqlCtx.read.load(filename,
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
df.show()

+---+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|_c0|      Date|Location|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustDir|WindGustSpeed|WindDir9am|WindDir3pm|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RISK_MM|RainTomorrow|
+---+----------+--------+-------+-------+--------+-----------+--------+-----------+-------------+----------+----------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+-------+------------+
|  0|01/12/2008|  Albury|   13.4|   22.9|     0.6|        3.2|     5.6|          W|         44.0|         W|       WNW|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|     8.0|     2.0|   16.9|   2

# Preprocessing

In [ ]:
df= df.drop('_c0', 'Date', 'Location', 'MinTemp', 'MaxTemp', 'Evaporation',
            'WindGustDir','WindDir9am', 'WindDir3pm', 'Humidity9am',
            'WindSpeed9am', 'WindSpeed3pm', 'Cloud9am', 'Temp9am', 'Temp3pm', 
            'RISK_MM', 'Pressure3pm', '')

df.printSchema()

root
 |-- Rainfall: double (nullable = true)
 |-- Sunshine: double (nullable = true)
 |-- WindGustSpeed: double (nullable = true)
 |-- Humidity3pm: double (nullable = true)
 |-- Pressure9am: double (nullable = true)
 |-- Cloud3pm: double (nullable = true)
 |-- RainToday: string (nullable = true)
 |-- RainTomorrow: string (nullable = true)



In [ ]:
df.show()

+--------+--------+-------------+-----------+-----------+--------+---------+------------+
|Rainfall|Sunshine|WindGustSpeed|Humidity3pm|Pressure9am|Cloud3pm|RainToday|RainTomorrow|
+--------+--------+-------------+-----------+-----------+--------+---------+------------+
|     0.6|     5.6|         44.0|       22.0|     1007.7|     2.0|       No|          No|
|     0.0|     5.6|         44.0|       25.0|     1010.6|     2.0|       No|          No|
|     0.0|     5.6|         46.0|       30.0|     1007.6|     2.0|       No|          No|
|     0.0|     5.6|         24.0|       16.0|     1017.6|     2.0|       No|          No|
|     1.0|     5.6|         41.0|       33.0|     1010.8|     8.0|       No|          No|
|     0.2|     5.6|         56.0|       23.0|     1009.2|     2.0|       No|          No|
|     0.0|     5.6|         50.0|       19.0|     1009.6|     2.0|       No|          No|
|     0.0|     5.6|         35.0|       19.0|     1013.4|     2.0|       No|          No|
|     0.0|

In [ ]:
from pyspark.sql.functions import col

'''df = df.where(col('Rainfall') != 'NA')
df = df.where(col('Sunshine') != 'NA')
df = df.where(col('WindGustSpeed') != 'NA')
df = df.where(col('Humidity3pm') != 'NA')
df = df.where(col('Pressure9am') != 'NA')
df = df.where(col('Cloud3pm') != 'NA')
df = df.where(col('RainToday') != 'NA')

df.show()'''

"df = df.where(col('Rainfall') != 'NA')\ndf = df.where(col('Sunshine') != 'NA')\ndf = df.where(col('WindGustSpeed') != 'NA')\ndf = df.where(col('Humidity3pm') != 'NA')\ndf = df.where(col('Pressure9am') != 'NA')\ndf = df.where(col('Cloud3pm') != 'NA')\ndf = df.where(col('RainToday') != 'NA')\n\ndf.show()"

In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType

df = df.na.drop()
df= df.withColumn('RainToday', when( (df.RainToday == 'No'), '0' ).otherwise('1') )
df= df.withColumn('RainTomorrow', when( (df.RainTomorrow == 'No'), '0' ).otherwise('1') )

def cast_df(df):
  df_casted = df.select(*(col(c).cast("float").alias(c) for c in df.columns))
  return df_casted

df = cast_df(df)
df = df.withColumn('RainToday' , df.RainToday.cast(IntegerType()) )
df = df.withColumn('RainTomorrow' , df.RainTomorrow.cast(IntegerType()) )

df.show()

+--------+--------+-------------+-----------+-----------+--------+---------+------------+
|Rainfall|Sunshine|WindGustSpeed|Humidity3pm|Pressure9am|Cloud3pm|RainToday|RainTomorrow|
+--------+--------+-------------+-----------+-----------+--------+---------+------------+
|     0.6|     5.6|         44.0|       22.0|     1007.7|     2.0|        0|           0|
|     0.0|     5.6|         44.0|       25.0|     1010.6|     2.0|        0|           0|
|     0.0|     5.6|         46.0|       30.0|     1007.6|     2.0|        0|           0|
|     0.0|     5.6|         24.0|       16.0|     1017.6|     2.0|        0|           0|
|     1.0|     5.6|         41.0|       33.0|     1010.8|     8.0|        0|           0|
|     0.2|     5.6|         56.0|       23.0|     1009.2|     2.0|        0|           0|
|     0.0|     5.6|         50.0|       19.0|     1009.6|     2.0|        0|           0|
|     0.0|     5.6|         35.0|       19.0|     1013.4|     2.0|        0|           0|
|     0.0|

In [ ]:
from pyspark.ml.feature import VectorAssembler

cols_in = [col for col in df.columns if col != 'RainTomorrow']

vecAssembler = VectorAssembler(inputCols = cols_in, outputCol="features")
vec_df= vecAssembler.transform( df )
vec_df= vec_df.select(['features', 'RainTomorrow'])

vec_df.show()

+--------------------+------------+
|            features|RainTomorrow|
+--------------------+------------+
|[0.60000002384185...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           0|
|[1.0,5.5999999046...|           0|
|[0.20000000298023...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           1|
|[1.39999997615814...|           0|
|[0.0,5.5999999046...|           1|
|[2.20000004768371...|           1|
|[15.6000003814697...|           1|
|[3.59999990463256...|           0|
|[0.0,5.5999999046...|           1|
|[16.7999992370605...|           1|
|[10.6000003814697...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           0|
|[0.0,5.5999999046...|           0|
+--------------------+------------+
only showing top 20 rows



In [ ]:
splits = vec_df.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

# MinMax scaler

In [ ]:
# minMax scaler
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

scalerModel = scaler.fit(train)
train_minMax_data = scalerModel.transform(train).select('scaledFeatures', 'RainTomorrow')
test_minMax_data = scalerModel.transform(test).select('scaledFeatures', 'RainTomorrow')

## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [x for x in range(1,30,5)])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=lr,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_minMax_data)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_minMax_data)
predictions.select('scaledFeatures', 'RainTomorrow', 'rawPrediction', 'prediction', 'probability').show(10)

+--------------------+------------+--------------------+----------+--------------------+
|      scaledFeatures|RainTomorrow|       rawPrediction|prediction|         probability|
+--------------------+------------+--------------------+----------+--------------------+
|[0.0,0.0,0.081300...|           1|[1.13809063430637...|       0.0|[0.75732890470780...|
|[0.0,0.0,0.081300...|           0|[0.68844038604561...|       0.0|[0.66561989310464...|
|[0.0,0.0,0.081300...|           1|[-0.6002061852778...|       1.0|[0.35429652324880...|
|[0.0,0.0,0.081300...|           1|[-0.3299224763974...|       1.0|[0.41825948596439...|
|[0.0,0.0,0.081300...|           1|[-0.1972316124512...|       1.0|[0.45085131859389...|
|[0.0,0.0,0.081300...|           1|[-0.2724601121432...|       1.0|[0.43230324106427...|
|[0.0,0.0,0.097560...|           0|[1.18333506692395...|       0.0|[0.76554692771981...|
|[0.0,0.0,0.097560...|           1|[0.32593563765098...|       0.0|[0.58077012750816...|
|[0.0,0.0,0.097560...

### Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7255593820131675


In [ ]:
from tabulate import tabulate

TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.828116 |
| Precision: | 0.626597 |
| Recall:    | 0.542349 |


## Decision tree

In [ ]:
from  pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [  x for x in range( 2, 15, 2 )   ])\
    .addGrid(dt.maxBins, [  x for x in range( 4, 32, 4 )   ])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=dt,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_minMax_data)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_minMax_data)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           0|       0.0|
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           1|       0.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           0|       1.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print('auc score: ', evaluator.evaluate(predictions)) 

auc score:  0.7157469113573618


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.843591 |
| Precision: | 0.711195 |
| Recall:    | 0.487361 |


## Random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [  x for x in range( 2, 15, 3 )   ]) \
    .addGrid(rf.numTrees, [  x for x in range( 2, 15, 3 )   ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=rf,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_minMax_data)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_minMax_data)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           0|       0.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           1|       0.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           0|       1.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7160644744179403


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.847268 |
| Precision: | 0.732926 |
| Recall:    | 0.481677 |


## Linear SVC

In [ ]:
from pyspark.ml.classification import LinearSVC

lin_svc = LinearSVC(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(   lin_svc.maxIter, [  x for x in range( 20, 200, 20 )   ]) \
    .addGrid(    lin_svc.fitIntercept, [    True,   False     ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=lin_svc,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_minMax_data)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_minMax_data)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           0|       0.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           1|       0.0|
|[0.0,0.0,0.081300...|           1|       1.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           1|       0.0|
|[0.0,0.0,0.097560...|           0|       0.0|
|[0.0,0.0,0.097560...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7050547986209558


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.843667 |
| Precision: | 0.731816 |
| Recall:    | 0.457431 |


# Standard Scaler

In [ ]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

scalerModel = scaler.fit(train)
train_scaledData = scalerModel.transform(train).select('scaledFeatures', 'RainTomorrow')
test_scaledData = scalerModel.transform(test).select('scaledFeatures', 'RainTomorrow')

## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [x for x in range(1,30,5)])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=lr,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_scaledData)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_scaledData)
predictions.select('scaledFeatures', 'RainTomorrow', 'rawPrediction', 'prediction', 'probability').show(10)

+--------------------+------------+--------------------+----------+--------------------+
|      scaledFeatures|RainTomorrow|       rawPrediction|prediction|         probability|
+--------------------+------------+--------------------+----------+--------------------+
|[0.0,0.0,1.252152...|           1|[1.13809063430544...|       0.0|[0.75732890470763...|
|[0.0,0.0,1.252152...|           0|[0.68844038604144...|       0.0|[0.66561989310372...|
|[0.0,0.0,1.252152...|           1|[-0.6002061852769...|       1.0|[0.35429652324902...|
|[0.0,0.0,1.252152...|           1|[-0.3299224763979...|       1.0|[0.41825948596428...|
|[0.0,0.0,1.252152...|           1|[-0.1972316124534...|       1.0|[0.45085131859335...|
|[0.0,0.0,1.252152...|           1|[-0.2724601121457...|       1.0|[0.43230324106366...|
|[0.0,0.0,1.399464...|           0|[1.18333506692205...|       0.0|[0.76554692771947...|
|[0.0,0.0,1.399464...|           1|[0.32593563764863...|       0.0|[0.58077012750759...|
|[0.0,0.0,1.399464...

### Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7255593820131675


In [ ]:
from tabulate import tabulate

TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.828116 |
| Precision: | 0.626597 |
| Recall:    | 0.542349 |


## Decision tree

In [ ]:
from  pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [  x for x in range( 2, 15, 2 )   ])\
    .addGrid(dt.maxBins, [  x for x in range( 4, 32, 4 )   ])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=dt,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_scaledData)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_scaledData)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           0|       0.0|
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           1|       0.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           0|       1.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print('auc score: ', evaluator.evaluate(predictions)) 

auc score:  0.7157469113573618


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.843591 |
| Precision: | 0.711195 |
| Recall:    | 0.487361 |


## Random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [  x for x in range( 2, 15, 3 )   ]) \
    .addGrid(rf.numTrees, [  x for x in range( 2, 15, 3 )   ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=rf,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_scaledData)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_scaledData)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           0|       0.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           1|       0.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           0|       1.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.713433259336582


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.843489 |
| Precision: | 0.714635 |
| Recall:    | 0.481097 |


## Linear SVC

In [ ]:
from pyspark.ml.classification import LinearSVC

lin_svc = LinearSVC(featuresCol = 'scaledFeatures', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(   lin_svc.maxIter, [  x for x in range( 20, 200, 20 )   ]) \
    .addGrid(    lin_svc.fitIntercept, [    True,   False     ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=lin_svc,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_scaledData)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_scaledData)
predictions.select('scaledFeatures', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|      scaledFeatures|RainTomorrow|prediction|
+--------------------+------------+----------+
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           0|       0.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           1|       0.0|
|[0.0,0.0,1.252152...|           1|       1.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           1|       0.0|
|[0.0,0.0,1.399464...|           0|       0.0|
|[0.0,0.0,1.399464...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7050547986209558


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.843667 |
| Precision: | 0.731816 |
| Recall:    | 0.457431 |


# PCA

In [ ]:
from pyspark.ml.feature import PCA

pca = PCA(k=3, inputCol= 'scaledFeatures', outputCol="pca")
model = pca.fit(train_scaledData)

train_pca = model.transform(train_scaledData).select('pca', 'RainTomorrow')
test_pca = model.transform(test_scaledData).select('pca', 'RainTomorrow')

## Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression(featuresCol = 'pca', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [x for x in range(1,30,5)])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=lr,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_pca)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_pca)
predictions.select('pca', 'RainTomorrow', 'rawPrediction', 'prediction', 'probability').show(10)

+--------------------+------------+--------------------+----------+--------------------+
|                 pca|RainTomorrow|       rawPrediction|prediction|         probability|
+--------------------+------------+--------------------+----------+--------------------+
|[31.8356910213303...|           1|[1.40417720291365...|       0.0|[0.80284591095222...|
|[31.2435766363801...|           0|[0.86530360893768...|       0.0|[0.70376753793948...|
|[30.1906031006671...|           1|[-0.4239243380324...|       1.0|[0.39557807291612...|
|[30.4007775377186...|           1|[-0.1514605098083...|       1.0|[0.46220709328052...|
|[30.4979699704505...|           1|[0.04893445248854...|       0.0|[0.51223117250879...|
|[30.4352620325116...|           1|[0.01993790250053...|       0.0|[0.50498431051265...|
|[31.8730760344107...|           0|[1.48587813571550...|       0.0|[0.81545879712088...|
|[30.9259474111435...|           1|[0.40149916899756...|       0.0|[0.59904779822410...|
|[30.9948597839834...

### Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.6775526979944906


In [ ]:
from tabulate import tabulate

TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.828499 |
| Precision: | 0.685654 |
| Recall:    | 0.407896 |


## Decision tree

In [ ]:
from  pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'pca', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [  x for x in range( 2, 15, 2 )   ])\
    .addGrid(dt.maxBins, [  x for x in range( 4, 32, 4 )   ])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=dt,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_pca)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_pca)
predictions.select('pca', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|                 pca|RainTomorrow|prediction|
+--------------------+------------+----------+
|[31.8356910213303...|           1|       0.0|
|[31.2435766363801...|           0|       0.0|
|[30.1906031006671...|           1|       1.0|
|[30.4007775377186...|           1|       1.0|
|[30.4979699704505...|           1|       1.0|
|[30.4352620325116...|           1|       1.0|
|[31.8730760344107...|           0|       0.0|
|[30.9259474111435...|           1|       0.0|
|[30.9948597839834...|           0|       0.0|
|[31.1466683068417...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print('auc score: ', evaluator.evaluate(predictions)) 

auc score:  0.7194910026331903


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.818131 |
| Precision: | 0.595201 |
| Recall:    | 0.543277 |


## Random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'pca', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [  x for x in range( 2, 15, 3 )   ]) \
    .addGrid(rf.numTrees, [  x for x in range( 2, 15, 3 )   ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=rf,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_pca)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_pca)
predictions.select('pca', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|                 pca|RainTomorrow|prediction|
+--------------------+------------+----------+
|[31.8356910213303...|           1|       0.0|
|[31.2435766363801...|           0|       0.0|
|[30.1906031006671...|           1|       1.0|
|[30.4007775377186...|           1|       1.0|
|[30.4979699704505...|           1|       1.0|
|[30.4352620325116...|           1|       1.0|
|[31.8730760344107...|           0|       0.0|
|[30.9259474111435...|           1|       0.0|
|[30.9948597839834...|           0|       0.0|
|[31.1466683068417...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7080634047318415


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.826022 |
| Precision: | 0.633521 |
| Recall:    | 0.497338 |


## Linear SVC

In [ ]:
from pyspark.ml.classification import LinearSVC

lin_svc = LinearSVC(featuresCol = 'pca', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(   lin_svc.maxIter, [  x for x in range( 20, 200, 20 )   ]) \
    .addGrid(    lin_svc.fitIntercept, [    True,   False     ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=lin_svc,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train_pca)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test_pca)
predictions.select('pca', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|                 pca|RainTomorrow|prediction|
+--------------------+------------+----------+
|[31.8356910213303...|           1|       0.0|
|[31.2435766363801...|           0|       0.0|
|[30.1906031006671...|           1|       1.0|
|[30.4007775377186...|           1|       0.0|
|[30.4979699704505...|           1|       0.0|
|[30.4352620325116...|           1|       0.0|
|[31.8730760344107...|           0|       0.0|
|[30.9259474111435...|           1|       0.0|
|[30.9948597839834...|           0|       0.0|
|[31.1466683068417...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.6658356846674781


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.827886 |
| Precision: | 0.703999 |
| Recall:    | 0.376341 |


# PCA con k = 10

In [ ]:
df = sqlCtx.read.load(filename,
                     format="csv", sep=",", inferSchema="true", header="true")

In [ ]:
# dropping of discrete attribute execept for RainToday and RainTomorrow
df = df.drop('_c0', 'Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RISK_MM')
df.show()

+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|   13.4|   22.9|     0.6|        3.2|     5.6|         44.0|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|     8.0|     2.0|   16.9|   21.8|       No|          No|
|    7.4|   25.1|     0.0|        3.2|     5.6|         44.0|         4.0|        22.0|       44.0|       25.0|     1010.6|     1007.8|     4.0|     2.0|   17.2|   24.3|       No|          No|
|   12.9|   25.7|     0.0|        3

In [ ]:
# preprocessing of RainToday and RainTomorrow attributes
from pyspark.sql.functions import when
from pyspark.sql.functions import regexp_replace
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col


df = df.na.drop()

df= df.withColumn('RainToday', when( (df.RainToday == 'No'), '0' ).otherwise('1') )
df= df.withColumn('RainTomorrow', when( (df.RainTomorrow == 'No'), '0' ).otherwise('1') )

def cast_df(df):
  df_casted = df.select(*(col(c).cast("float").alias(c) for c in df.columns))

  return df_casted

df = cast_df(df)

df = df.withColumn('RainToday' , df.RainToday.cast(IntegerType()) )
df = df.withColumn('RainTomorrow' , df.RainTomorrow.cast(IntegerType()) )

df.show()

+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|MinTemp|MaxTemp|Rainfall|Evaporation|Sunshine|WindGustSpeed|WindSpeed9am|WindSpeed3pm|Humidity9am|Humidity3pm|Pressure9am|Pressure3pm|Cloud9am|Cloud3pm|Temp9am|Temp3pm|RainToday|RainTomorrow|
+-------+-------+--------+-----------+--------+-------------+------------+------------+-----------+-----------+-----------+-----------+--------+--------+-------+-------+---------+------------+
|   13.4|   22.9|     0.6|        3.2|     5.6|         44.0|        20.0|        24.0|       71.0|       22.0|     1007.7|     1007.1|     8.0|     2.0|   16.9|   21.8|        0|           0|
|    7.4|   25.1|     0.0|        3.2|     5.6|         44.0|         4.0|        22.0|       44.0|       25.0|     1010.6|     1007.8|     4.0|     2.0|   17.2|   24.3|        0|           0|
|   12.9|   25.7|     0.0|        3

In [ ]:
from pyspark.ml.feature import VectorAssembler

cols_in = [col for col in df.columns if col != 'RainTomorrow']

vecAssembler = VectorAssembler(inputCols = cols_in, outputCol="features")
vec_df= vecAssembler.transform( df )
vec_df= vec_df.select(['features', 'RainTomorrow'])

vec_df.show()

+--------------------+------------+
|            features|RainTomorrow|
+--------------------+------------+
|[13.3999996185302...|           0|
|[7.40000009536743...|           0|
|[12.8999996185302...|           0|
|[9.19999980926513...|           0|
|[17.5,32.29999923...|           0|
|[14.6000003814697...|           0|
|[14.3000001907348...|           0|
|[7.69999980926513...|           0|
|[9.69999980926513...|           1|
|[13.1000003814697...|           0|
|[13.3999996185302...|           1|
|[15.8999996185302...|           1|
|[15.8999996185302...|           1|
|[12.6000003814697...|           0|
|[14.1000003814697...|           1|
|[13.5,22.89999961...|           1|
|[11.1999998092651...|           0|
|[9.80000019073486...|           0|
|[11.5,29.29999923...|           0|
|[17.1000003814697...|           0|
+--------------------+------------+
only showing top 20 rows



In [ ]:
splits = vec_df.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1]

In [ ]:
from pyspark.ml.feature import PCA

pca = PCA(k=10, inputCol= 'features', outputCol="pca")
model = pca.fit(train)

train = model.transform(train).select('pca', 'RainTomorrow')
test = model.transform(test).select('pca', 'RainTomorrow')

## Logistic regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

lr = LogisticRegression(featuresCol = 'pca', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(lr.maxIter, [x for x in range(1,30,5)])\
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=lr,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test)
predictions.select('pca', 'RainTomorrow', 'rawPrediction', 'prediction', 'probability').show(10)

+--------------------+------------+--------------------+----------+--------------------+
|                 pca|RainTomorrow|       rawPrediction|prediction|         probability|
+--------------------+------------+--------------------+----------+--------------------+
|[212.536406921154...|           0|[-0.3546431382136...|       1.0|[0.41225692683150...|
|[207.093535205613...|           0|[-2.5964311827991...|       1.0|[0.06936845682363...|
|[188.600061171171...|           0|[0.99853565488069...|       0.0|[0.73077057351024...|
|[210.248739397244...|           1|[-1.3907965886769...|       1.0|[0.19928061645456...|
|[155.360906181435...|           0|[3.66729979210278...|       0.0|[0.97509095569948...|
|[150.967380876919...|           0|[4.24557067422575...|       0.0|[0.98587482425290...|
|[156.245794656182...|           0|[1.54641812235209...|       0.0|[0.82439579520435...|
|[209.325058471838...|           0|[-1.4731502644112...|       1.0|[0.18646426145206...|
|[193.569310970093...

### Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7159505157749048


In [ ]:
from tabulate import tabulate

TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.84381  |
| Precision: | 0.716608 |
| Recall:    | 0.486588 |


## Decision Tree

In [ ]:
from  pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol = 'pca', labelCol = 'RainTomorrow')

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(dt.maxDepth, [  x for x in range( 2, 15, 2 )   ])\
    .addGrid(dt.maxBins, [  x for x in range( 4, 32, 4 )   ])\
    .build()

# cross-validation
crossval = CrossValidator(estimator=dt,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test)
predictions.select('pca', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|                 pca|RainTomorrow|prediction|
+--------------------+------------+----------+
|[154.930658591095...|           0|       0.0|
|[203.256941358130...|           0|       0.0|
|[153.257078385716...|           0|       0.0|
|[155.380046565651...|           0|       0.0|
|[187.474803445247...|           0|       0.0|
|[198.055233227185...|           0|       0.0|
|[164.782755066898...|           0|       0.0|
|[152.381311388754...|           0|       0.0|
|[152.140399741360...|           0|       0.0|
|[198.416404049839...|           0|       1.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7159505157749048


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.84381  |
| Precision: | 0.716608 |
| Recall:    | 0.486588 |


## Random forest

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(featuresCol = 'pca', labelCol = 'RainTomorrow', seed= 42)

# grid search
paramGrid = ParamGridBuilder() \
    .addGrid(rf.maxDepth, [  x for x in range( 2, 15, 3 )   ]) \
    .addGrid(rf.numTrees, [  x for x in range( 2, 15, 3 )   ]) \
    .build()

# cross-validation
crossval = CrossValidator(estimator=rf,
                          numFolds= 5,
                           estimatorParamMaps=paramGrid,
                           evaluator=BinaryClassificationEvaluator(labelCol='RainTomorrow',
                                                                   rawPredictionCol='prediction',
                                                                   metricName="areaUnderROC"))

cvModel = crossval.fit(train)
bestModel = cvModel.bestModel

In [ ]:
predictions = bestModel.transform(test)
predictions.select('pca', 'RainTomorrow', 'prediction').show(10)

+--------------------+------------+----------+
|                 pca|RainTomorrow|prediction|
+--------------------+------------+----------+
|[212.536406921154...|           0|       0.0|
|[207.093535205613...|           0|       1.0|
|[188.600061171171...|           0|       0.0|
|[210.248739397244...|           1|       1.0|
|[155.360906181435...|           0|       0.0|
|[150.967380876919...|           0|       0.0|
|[156.245794656182...|           0|       0.0|
|[209.325058471838...|           0|       1.0|
|[193.569310970093...|           0|       0.0|
|[156.139212278597...|           0|       0.0|
+--------------------+------------+----------+
only showing top 10 rows



### Evaluation

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                                labelCol='RainTomorrow',
                                                metricName="areaUnderROC")
print( evaluator.evaluate(predictions)) 

0.7189139681479122


In [ ]:
TP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 1)).count() +0.1 # evita ZeroDivisionError
FP = predictions.filter((predictions['prediction'] == 1.0) & (predictions['RainTomorrow'] == 0)).count()
TN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 0)).count()
FN = predictions.filter((predictions['prediction'] == 0.0) & (predictions['RainTomorrow'] == 1)).count()



print(tabulate([['Accuracy:', (TP+TN)/(TP+TN+FP+FN)], 
                    ['Precision:', TP/(TP+FP)], 
                    ['Recall:', TP/(TP+FN)]], tablefmt='orgtbl'))

| Accuracy:  | 0.846793 |
| Precision: | 0.729102 |
| Recall:    | 0.489517 |
